In [ ]:
%%bash

pip install -q kaggle
mkdir -p ~/.kaggle
cp drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
kaggle datasets download -d abdelghaniaaba/wildfire-prediction-dataset
unzip -qq wildfire-prediction-dataset.zip

100%|██████████| 1.45G/1.45G [01:08<00:00, 22.6MB/s]


# Wildfire prediction dataset

In [ ]:
import tensorflow as tf

from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import pandas as pd

In [ ]:
train_path = "train"
valid_path = "valid"
test_path = "test"
image_shape = (350,350,3)
N_CLASSES = 2
BATCH_SIZE = 256

train_datagen = ImageDataGenerator(dtype='float32', rescale= 1./255.)
train_generator = train_datagen.flow_from_directory(train_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')

valid_datagen = ImageDataGenerator(dtype='float32', rescale= 1./255.)
valid_generator = valid_datagen.flow_from_directory(valid_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')

test_datagen = ImageDataGenerator(dtype='float32', rescale = 1.0/255.0)
test_generator = test_datagen.flow_from_directory(test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [ ]:
weight_decay = 1e-3
first_model = Sequential([
    Conv2D(filters = 8 , kernel_size = 2, activation = 'relu', 
    input_shape = image_shape), MaxPooling2D(pool_size = 2),
    
    Conv2D(filters = 16 , kernel_size = 2, activation = 'relu', 
    input_shape = image_shape), MaxPooling2D(pool_size = 2),
    
    Conv2D(filters = 32 , kernel_size = 2, activation = 'relu',
           kernel_regularizer = regularizers.l2(weight_decay)),
    MaxPooling2D(pool_size = 2),
    
    Dropout(0.4),
    Flatten(),
    Dense(300,activation='relu'),
    Dropout(0.5),
    Dense(2,activation='softmax')
])
first_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 349, 349, 8)       104       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 174, 174, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 173, 173, 16)      528       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 86, 86, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 85, 85, 32)        2080      
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 42, 42, 32)      

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

checkpointer = ModelCheckpoint('first_model.hdf5',verbose=1, save_best_only= True)
early_stopping = EarlyStopping(monitor= 'val_loss', patience= 10)
optimizer = optimizers.Adam(learning_rate= 0.00001, decay= 1e-5)
first_model.compile(loss= 'categorical_crossentropy', optimizer= optimizer,
                    metrics=['AUC','acc'])

# TRAIN
history = first_model.fit(train_generator,
                    epochs = 50,
                    verbose = 1,
                    validation_data = valid_generator,
                    callbacks = [checkpointer, early_stopping])

Epoch 1/50
119/119 [==============================] - ETA: 0s - loss: 0.6278 - auc: 0.7145 - acc: 0.6110
Epoch 1: val_loss improved from inf to 0.54070, saving model to first_model.hdf5
119/119 [==============================] - 208s 2s/step - loss: 0.6278 - auc: 0.7145 - acc: 0.6110 - val_loss: 0.5407 - val_auc: 0.8949 - val_acc: 0.8249
Epoch 2/50
119/119 [==============================] - ETA: 0s - loss: 0.4833 - auc: 0.8974 - acc: 0.8231
Epoch 2: val_loss improved from 0.54070 to 0.41581, saving model to first_model.hdf5
119/119 [==============================] - 206s 2s/step - loss: 0.4833 - auc: 0.8974 - acc: 0.8231 - val_loss: 0.4158 - val_auc: 0.9322 - val_acc: 0.8481
Epoch 3/50
119/119 [==============================] - ETA: 0s - loss: 0.3945 - auc: 0.9344 - acc: 0.8556
Epoch 3: val_loss improved from 0.41581 to 0.34597, saving model to first_model.hdf5
119/119 [==============================] - 205s 2s/step - loss: 0.3945 - auc: 0.9344 - acc: 0.8556 - val_loss: 0.3460 - val_au

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
test_image = image.load_img("/content/test/nowildfire/-113.91777,50.901087.jpg", target_size=(256, 256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = first_model.predict(test_image)
print(train_generator.class_indices)
print(result)